## Deliverable 2. Create a Customer Travel Destinations Map.

### Create The Preferred Cities DataFrame and Gather Hotel Info

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Lat,Lng
0,0,Qaanaaq,-7.94,51,92,8.77,overcast clouds,GL,77.4840,-69.3632
1,1,Bluff,49.80,78,99,10.89,overcast clouds,NZ,-46.6000,168.3333
2,2,Atar,82.15,9,29,10.36,scattered clouds,MR,20.5169,-13.0499
3,3,Fairbanks,5.34,63,100,3.44,overcast clouds,US,64.8378,-147.7164
4,4,Iskateley,-2.16,99,92,4.72,overcast clouds,RU,67.6803,53.1512


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Lat,Lng
2,2,Atar,82.15,9,29,10.36,scattered clouds,MR,20.5169,-13.0499
10,10,Kpandae,94.57,9,1,6.87,clear sky,GH,8.4689,-0.0113
12,12,Rikitea,77.49,74,100,9.64,overcast clouds,PF,-23.1203,-134.9692
17,17,Arraial Do Cabo,80.46,83,40,5.75,scattered clouds,BR,-22.9661,-42.0278
18,18,Avarua,78.85,83,20,6.91,few clouds,CK,-21.2078,-159.7750
...,...,...,...,...,...,...,...,...,...,...
685,685,Tacoronte,70.12,53,0,15.99,clear sky,ES,28.4769,-16.4102
687,687,Manggar,80.71,79,43,8.52,scattered clouds,ID,-2.8833,108.2667
690,690,Grand Gaube,81.37,81,100,8.99,overcast clouds,MU,-20.0064,57.6608
691,691,Narasannapeta,73.35,82,0,11.48,clear sky,IN,18.4167,84.0500


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                270
City                   270
Max Temp               270
Humidity               270
Cloudiness             270
Wind Speed             270
Current Description    270
Country                267
Lat                    270
Lng                    270
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                267
City                   267
Max Temp               267
Humidity               267
Cloudiness             267
Wind Speed             267
Current Description    267
Country                267
Lat                    267
Lng                    267
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atar,MR,82.15,scattered clouds,20.5169,-13.0499,
10,Kpandae,GH,94.57,clear sky,8.4689,-0.0113,
12,Rikitea,PF,77.49,overcast clouds,-23.1203,-134.9692,
17,Arraial Do Cabo,BR,80.46,scattered clouds,-22.9661,-42.0278,
18,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
19,Tessalit,ML,75.51,clear sky,20.1986,1.0114,
20,Camocim,BR,82.40,overcast clouds,-2.9022,-40.8411,
26,Caravelas,BR,80.31,overcast clouds,-17.7125,-39.2481,
27,Ahipara,NZ,71.02,broken clouds,-35.1667,173.1667,
31,Busselton,AU,73.26,clear sky,-33.6500,115.3333,


### Find Hotels Nearby

In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping!')
        

Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!
Hotel not found...skipping!


In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   267
Country                267
Max Temp               267
Current Description    267
Lat                    267
Lng                    267
Hotel Name             267
dtype: int64

In [9]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_Vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Generate a Map with Pop-Ups and Marker Layer

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))